In [58]:
# Instalación del docker ollama
# Con CPU
# docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
# COn GPU NVIDIA
# docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

# Descargar y ejecutar el modelo llama3
# docker exec -it ollama ollama run llama3

# Despues solo se ejecuta este comando y ya esta montado el ollama
# docker start ollama

| Model                | Parameters | Size  | Download Command         |
|----------------------|------------|-------|--------------------------|
| Llama 3              | 8B         | 4.7GB | ollama run llama3        |
| Llama 3              | 70B        | 40GB  | ollama run llama3:70b    |
| Phi-3                | 3.8B       | 2.3GB | ollama run phi3          |
| Mistral              | 7B         | 4.1GB | ollama run mistral       |
| Neural Chat          | 7B         | 4.1GB | ollama run neural-chat   |
| Starling             | 7B         | 4.1GB | ollama run starling-lm   |
| Code Llama           | 7B         | 3.8GB | ollama run codellama     |
| Llama 2 Uncensored   | 7B         | 3.8GB | ollama run llama2-uncensored |
| LLaVA                | 7B         | 4.5GB | ollama run llava         |
| Gemma                | 2B         | 1.4GB | ollama run gemma:2b      |
| Gemma                | 7B         | 4.8GB | ollama run gemma:7b      |
| Solar                | 10.7B      | 6.1GB | ollama run solar         |

In [59]:
#Instalación de las librerias necesarias de Ollama
#!pip install ollama
#Intalacion de las libreria de LangChain
#!pip install langchain


In [60]:
import ollama

from langchain_community.llms import Ollama

llm = Ollama(model="llama3", temperature=0)

In [61]:
respuesta = llm.invoke("Codigo de python que calcule el factorial de un numero")

In [62]:
print(respuesta)

Aquí te dejo un ejemplo de código en Python que calcula el factorial de un número:
```
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n-1)

num = int(input("Introduce un número: "))
print(f"El factorial de {num} es {factorial(num)}")
```
Este código define una función `factorial` que toma un parámetro `n` y devuelve el factorial de ese número. La función se define de manera recursiva, lo que significa que llama a sí misma con un valor menor hasta que llega al caso base (en este caso, cuando `n` es 0 o 1).

La función se utiliza para calcular el factorial de un número introducido por el usuario.

Puedes ejecutar este código en una ventana de comandos o en un entorno de desarrollo integrado como PyCharm y obtener el resultado del factorial de un número que introduces.

Por ejemplo, si introduces el número 5, el programa imprimirá:
```
El factorial de 5 es 120
```
Espero que esto te ayude. ¡Si tienes alguna pregunta o necesitas ayuda adi

In [63]:
#Intalamos la libreria de cargar PDF
#!pip install pypdf

In [64]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
import ollama

In [65]:
loader = PyPDFLoader('estatuto.pdf')
paginas = loader.load()

In [66]:
len(paginas)

25

In [67]:
print(paginas[4].page_content)

TEXTO CONSOLIDADO
Última modificación: 24 de marzo de 2021
DON JUAN CARLOS I,
REY DE ESPAÑA
A todos los que la presente vieren y entendieren,
Sabed: Que las Cortes Generales han aprobado y Yo vengo en sancionar la siguiente 
Ley Orgánica:
PREÁMBULO
Cantabria, como comunidad histórica perfectamente definida dentro de España y 
haciendo uso del derecho a la autonomía que la Constitución reconoce en su Título VIII y en 
base a las decisiones de la Diputación Provincial y de sus Ayuntamientos libre y 
democráticamente expresadas, manifiesta su voluntad de constituirse en Comunidad 
Autónoma de acuerdo con lo dispuesto en el artículo 143 de la Constitución.
El presente Estatuto es la expresión jurídica de la identidad de Cantabria y define sus 
instituciones, competencias y recursos, dentro de la indisoluble unidad de España y en el 
marco de la más estrecha solidaridad con las demás nacionalidades y regiones.
Cantabria encuentra en sus instituciones la voluntad de respetar los derechos 
fu

In [68]:
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=1024,
    chunk_overlap=128,
    length_function=len
)


In [69]:
docs = text_splitter.split_documents(paginas)

In [70]:
len(docs), len(paginas)

(91, 25)

In [71]:
#Intalamos la libreria de chromadb
# Se tiene que instalar https://visualstudio.microsoft.com/es/visual-cpp-build-tools/
#!pip install chromadb

In [72]:
embeddings = OllamaEmbeddings(model="llama3")


In [73]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [74]:
def ollama_llm(question, context):
    """
    Realiza una consulta a un modelo de lenguaje de aprendizaje automático (LLM) llamado Ollama.
    
    Parameters:
        question (str): La pregunta que se desea realizar al modelo.
        context (str): El contexto en el que se basa la pregunta.
        
    Returns:
        str: La respuesta generada por el modelo de lenguaje.
    """
    formatted_prompt = f"Pregunta: {question}\n\nContexto (responde solo sobre el contenido del texto entregado): {context}"
    response = ollama.chat(model="llama3", messages=[{'role': 'user', 'content': formatted_prompt}],
                           options={'temperature': 0})
    
    return response['message']['content']

    

In [75]:
retriever = vectorstore.as_retriever()

def combine_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

In [76]:
def rag_chains(question, docs):
    retrieved_docs = retriever.invoke(question)
    formatted_context = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)


In [81]:
result = rag_chains("resumen del Artículo 17", docs)
result

'El artículo 17 del texto consolidado no existe, ya que el texto entregado es un boletín oficial del Estado que contiene varias leyes y disposiciones, pero no hay un artículo 17 específico. Sin embargo, se puede resumir lo siguiente:\n\n* El artículo 3 describe las condiciones en las que el Presidente del Gobierno no puede acordar la disolución del Parlamento: durante el primer período de sesiones de la legislatura cuando reste menos de un año para su terminación, cuando se encuentre en tramitación una moción de censura, o antes de que transcurra un año desde la última disolución por este procedimiento.\n* El artículo final (no numerado) describe el cambio de denominación de la provincia de Santander por provincia de Cantabria y establece que el Gobierno dictará las disposiciones oportunas para dar cumplimiento a las consecuencias derivadas de esta disposición en un plazo de un año.'